In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from numba import jit
import pandas as pd
import numpy as np
import math
import random

In [3]:
df = pd.read_csv('./kendaraan_train_clean.csv')
df.head()

,id,Umur,SIM,Kode_Daerah,Sudah_Asuransi,Umur_Kendaraan,Kendaraan_Rusak,Premi,Kanal_Penjualan,Lama_Berlangganan,Tertarik
0,1,30,1,33,1,< 1 Tahun,0,28029,152,97,0
1,2,48,1,39,0,> 2 Tahun,1,25800,29,158,0
2,4,58,1,48,0,1-2 Tahun,0,2630,124,63,0
3,6,21,1,35,1,< 1 Tahun,0,22735,152,171,0
4,9,20,1,8,1,< 1 Tahun,0,30786,160,31,0


In [4]:
df.describe()

,id,Umur,SIM,Kode_Daerah,Sudah_Asuransi,Kendaraan_Rusak,Premi,Kanal_Penjualan,Lama_Berlangganan,Tertarik
count,171068.000000,171068.000000,171068.000000,171068.000000,171068.000000,171068.000000,171068.000000,171068.000000,171068.000000,171068.000000
mean,142794.020729,38.818137,0.997931,26.411334,0.459794,0.504443,30506.747282,112.260206,154.253063,0.121577
std,82491.716355,15.531695,0.045443,13.262377,0.498382,0.499982,17090.409733,54.106712,83.862579,0.326798
min,1.000000,20.000000,0.000000,0.000000,0.000000,0.000000,2630.000000,1.000000,10.000000,0.000000
25%,71350.750000,25.000000,1.000000,15.000000,0.000000,0.000000,24369.750000,29.000000,81.000000,0.000000
50%,142768.000000,36.000000,1.000000,28.000000,0.000000,1.000000,31603.000000,140.000000,154.000000,0.000000
75%,214070.250000,49.000000,1.000000,35.000000,1.000000,1.000000,39323.250000,152.000000,227.000000,0.000000
max,285831.000000,85.000000,1.000000,52.000000,1.000000,1.000000,540165.000000,163.000000,299.000000,1.000000


In [5]:
@jit
def euclidean(x, y):
    d = math.sqrt(sum((x - y)**2))
    return d

In [6]:
@jit
def kmeans(k, max_iter, df):
    
    centroid = {}
    
    for c in range(k):
        ran = random.randint(0, len(df))
        centroid[c] = df[ran]

    for _ in range(max_iter):
        cluster = {}
        
        for i in range(k):
            cluster[i] = []
            
        for x in df:           
            dist = [euclidean(x, centroid[c]) for c in centroid]
            cluster[dist.index(min(dist))].append(x)
            
        for cl in cluster:
            centroid[cl] = np.mean(cluster[cl], axis = 0)

    return centroid, cluster

In [ ]:
data = df.to_numpy()

# Memasukkan Nilai K dan Maksimum Iterasi
k = 5
max_iter = 1000

# Memanggil Fungsi kmeans Yang Akan Mengembalikan Nilai Centroid Dan Cluster
centroid, cluster = kmeans(k, max_iter, data)